<div align="right"><i>Peter Norvig<br>Sept 25, 2024</i></div>

# The Languages of English, Math, and Programming

My colleague [Wei-Hwa Huang](https://en.wikipedia.org/wiki/Wei-Hwa_Huang) posed the following problem to several AI large language model (LLM) chatbots: 

- **List all the ways in which three distinct positive integers have a product of 108.**

All the LLMs he tried failed. I reran the experiment on more LLMs (and a human), and a few of them succeeded. I thought they might do better with this prompt:

- **Write a Python program to list all the ways in which three distinct positive integers have a product of 108.**



# TLDR: Conclusions

Only 2 of the  9 LLMs solved the "list all ways" prompt, but 7 out of 9 solved the "write a program" prompt.  **The language that a problem-solver uses matters!** Sometimes a natural language such as English is a good choice, sometimes you need the language of mathematical equations, or chemical equations, or musical notation, and sometimes a programming language is best. Written language is an amazing invention that has enabled human culture to build over the centuries (and also enabled LLMs to work). But human ingenuity has divised other notations that are more specialized but very effective in limited domains.

Some more notes on the "write a program" prompt:


- The LLMs all started their answer  by stating that 108 = 2 × 2 × 3 × 3 × 3, and then tried to partition those factors into three distinct subsets and report all ways to do so.
- So far so good!
- But some of them forgot that 1 could be a factor of 108 (or equivalently, that the empty set of factors is a valid subset).
- Some of them only forgot the triplets (1, 2, 54) and (1, 3, 36), but somehow got (1, 4, 27) and (1, 6, 18).
  - Perhaps the forgetting was because their attention mechanism didn't go back far enough?
- The models might have skipped 1 as a factor because 1 is not listed in the prime factorization, so it is easy to forget. But in programming, it is more natural to run a loop from 1 to *n* than from 2 to *n*; that's why I tried the "write a program" prompt.
- Some of the models ignored the need for "distinct" integers, and proposed, (3, 6, 6) or (1, 108, 1).
- Perplexity proposed (2, 4, 13.5)  on the first run, but on a rerun proposed and then eliminated 13.5 to get the correct result.

Summary of which solver solved which problems:

|Solver|"List all ways"|"Write a program"|
|--|--|--|
|[A human programmer](https://github.com/norvig/)|yes|yes|
|[Gemini Advanced](https://gemini.google.com/app)|no (4 + 1 nondistinct)|yes|
|[ChatGPT 4o](https://chatgpt.com/)|**yes**|yes|
|[Microsoft Copilot](https://copilot.microsoft.com/)|no (6/8)|yes|
|[Anthropic Claude 3.5 Sonnet](https://claude.ai/new)|no (6/8)|yes|
|[Meta AI Llama 3](https://www.meta.ai/)|no (6/8)|**no** (extra permutations)|
|[Perplexity](https://www.perplexity.ai/)|**yes**|yes|
|[Cohere Chat](https://cohere.com/chat)|no (8 + 2 nondistinct)|**no** (0/8)|
|[HuggingFace Chat](https://huggingface.co/chat/)|no (8 + 1 nondistinct)|yes|
|[You.com](https://you.com/)|no (6/8)|yes|
|**Total of LLMs**|**2/9 yes**|**7/9 yes**|


Below are the programs produced by all the solvers:

# Human

A human (me) generated this correct solution:

In [1]:
from math      import prod
from itertools import combinations
from typing    import *

def find_products(k=3, N=108) -> Set[Tuple[int, ...]]:
    """A list of all ways in which `k` distinct positive integers have a product of `N`.""" 
    factors = {i for i in range(1, N + 1) if N % i == 0}
    return {ints for ints in combinations(factors, k) if prod(ints) == N}

find_products()

{(1, 2, 54),
 (1, 3, 36),
 (1, 4, 27),
 (1, 6, 18),
 (1, 9, 12),
 (2, 3, 18),
 (2, 6, 9),
 (3, 4, 9)}

The program can solve different versions of the problem (although it would be slow when `n` is over a billion or so):

In [2]:
find_products(5, 360)

{(1, 2, 3, 4, 15),
 (1, 2, 3, 5, 12),
 (1, 2, 3, 6, 10),
 (1, 2, 4, 5, 9),
 (1, 3, 4, 5, 6)}

# Gemini Advanced

Gemini produced three drafts, of which the following one was correct. In another draft, it had the line `k = product // (i * j)`, using integer division, which is incompatible with the `k.is_integer()` test (maybe `int` should support `.is_integer`?). Here is the correct draft:

In [3]:
def find_combinations(product):
  """Finds all combinations of three distinct positive integers that multiply to the given product.

  Args:
    product: The product to find combinations for.

  Returns:
    A list of tuples, where each tuple represents a combination of three integers.
  """

  combinations = []
  for i in range(1, product):
    for j in range(i + 1, product):
      k = product / (i * j)
      if k > j and k.is_integer():
        combinations.append((i, j, int(k)))
  return combinations

product = 108
result = find_combinations(product)
print(result)

[(1, 2, 54), (1, 3, 36), (1, 4, 27), (1, 6, 18), (1, 9, 12), (2, 3, 18), (2, 6, 9), (3, 4, 9)]


# ChatGPT 4o

ChatGPT got it right (although it imported `itertools` and then never used it).

In [4]:
import itertools

def find_triplets_with_product(product):
    solutions = []
    
    # Iterate over all possible combinations of distinct integers
    for a in range(1, int(product ** (1/3)) + 1):
        for b in range(a + 1, product // a + 1):
            for c in range(b + 1, product // (a * b) + 1):
                if a * b * c == product:
                    solutions.append((a, b, c))
    
    return solutions

product = 108
triplets = find_triplets_with_product(product)

if triplets:
    print(f"All distinct triplets of integers that multiply to {product}:")
    for triplet in triplets:
        print(triplet)
else:
    print(f"No distinct triplets found that multiply to {product}.")


All distinct triplets of integers that multiply to 108:
(1, 2, 54)
(1, 3, 36)
(1, 4, 27)
(1, 6, 18)
(1, 9, 12)
(2, 3, 18)
(2, 6, 9)
(3, 4, 9)


# Microsoft CoPilot

Bing produces a very clean (but somewhat slower) `find_triplets` function.

In [5]:
def find_triplets(product):
    triplets = []
    for a in range(1, product + 1):
        for b in range(a + 1, product + 1):
            for c in range(b + 1, product + 1):
                if a * b * c == product:
                    triplets.append((a, b, c))
    return triplets

product = 108
triplets = find_triplets(product)

print(f"All distinct triplets with a product of {product}:")
for triplet in triplets:
    print(triplet)

All distinct triplets with a product of 108:
(1, 2, 54)
(1, 3, 36)
(1, 4, 27)
(1, 6, 18)
(1, 9, 12)
(2, 3, 18)
(2, 6, 9)
(3, 4, 9)


# Claude 3.5 Sonnet

Claude's answer is quite simlar to ChatGPT 4o's (but I would criticize it for not taking a parameter):

In [6]:
def find_combinations():
    result = []
    for i in range(1, 108):
        for j in range(i + 1, 108):
            k = 108 // (i * j)
            if i * j * k == 108 and k > j:
                result.append((i, j, k))
    return result

def main():
    combinations = find_combinations()
    print(f"There are {len(combinations)} ways to express 108 as a product of three distinct positive integers:")
    for combo in combinations:
        print(f"{combo[0]} x {combo[1]} x {combo[2]} = 108")

if __name__ == "__main__":
    main()

There are 8 ways to express 108 as a product of three distinct positive integers:
1 x 2 x 54 = 108
1 x 3 x 36 = 108
1 x 4 x 27 = 108
1 x 6 x 18 = 108
1 x 9 x 12 = 108
2 x 3 x 18 = 108
2 x 6 x 9 = 108
3 x 4 x 9 = 108


# Meta AI Llama 3.1

This program forgets to test for `i < j < k`, and thus produces six permutations of each triplet. Oddly, it claims that "*When you run this program, it will output: [(1, 2, 54), (1, 3, 36), (1, 4, 27), (1, 6, 18), (1, 9, 12), (2, 3, 18), (2, 6, 9), (3, 4, 9)]*", so it knows the correct answer, but it didn't run the program to test that it is correct.

In [7]:
def find_factors(n):
    factors = []
    for i in range(1, n + 1):
        if n % i == 0:
            factors.append(i)
    return factors

def find_triplets(n):
    factors = find_factors(n)
    triplets = []
    for i in factors:
        for j in factors:
            if i * j <= n:
                k = n // (i * j)
                if i != j and j != k and i != k and i * j * k == n:
                    triplets.append((i, j, k))
    return triplets

print(find_triplets(108))

[(1, 2, 54), (1, 3, 36), (1, 4, 27), (1, 6, 18), (1, 9, 12), (1, 12, 9), (1, 18, 6), (1, 27, 4), (1, 36, 3), (1, 54, 2), (2, 1, 54), (2, 3, 18), (2, 6, 9), (2, 9, 6), (2, 18, 3), (2, 54, 1), (3, 1, 36), (3, 2, 18), (3, 4, 9), (3, 9, 4), (3, 18, 2), (3, 36, 1), (4, 1, 27), (4, 3, 9), (4, 9, 3), (4, 27, 1), (6, 1, 18), (6, 2, 9), (6, 9, 2), (6, 18, 1), (9, 1, 12), (9, 2, 6), (9, 3, 4), (9, 4, 3), (9, 6, 2), (9, 12, 1), (12, 1, 9), (12, 9, 1), (18, 1, 6), (18, 2, 3), (18, 3, 2), (18, 6, 1), (27, 1, 4), (27, 4, 1), (36, 1, 3), (36, 3, 1), (54, 1, 2), (54, 2, 1)]


# Perplexity

Perplexity does well.

In [8]:
def find_combinations(target):
    combinations = []
    
    for i in range(1, target + 1):
        if target % i == 0:
            for j in range(i + 1, target + 1):
                if (target // i) % j == 0:
                    k = target // (i * j)
                    if k > j and i * j * k == target:
                        combinations.append((i, j, k))
    
    return combinations

target_product = 108
result = find_combinations(target_product)

print(f"All combinations of three distinct positive integers with a product of {target_product}:")
for combo in result:
    print(f"{combo[0]} x {combo[1]} x {combo[2]} = {target_product}")

print(f"\nTotal number of combinations: {len(result)}")

All combinations of three distinct positive integers with a product of 108:
1 x 2 x 54 = 108
1 x 3 x 36 = 108
1 x 4 x 27 = 108
1 x 6 x 18 = 108
1 x 9 x 12 = 108
2 x 3 x 18 = 108
2 x 6 x 9 = 108
3 x 4 x 9 = 108

Total number of combinations: 8


# Cohere Chat Playground

Cohere fails to find any combinations. The 5th line should just be `k = product // i // j`; Cohere bizarrely adds `* j * i`. It also fails to test that `i < j < k`.

In [9]:
def find_combinations(product):
    combinations = []
    for i in range(1, product // 3 + 1):
        for j in range(i, product // 2 + 1):
            k = product // i // j * j * i
            if i * j * k == product and i != j and j != k and i != k:
                combinations.append((i, j, k))
    return combinations

product = 108
combinations = find_combinations(product)
print(f"All combinations of three distinct positive integers with a product of {product}:")
for combo in combinations:
    print(combo)

All combinations of three distinct positive integers with a product of 108:


# HuggingChat

Hugging Chat produced a correct concise program. I note that `i < j < k` would be cleaner than `k >= j and i!= j and j!= k` here (and for others as well).

In [10]:
def find_triplets(n):
    triplets = []
    for i in range(1, n):
        for j in range(i+1, n):
            k = n // (i * j)
            if k >= j and i * j * k == n and i!= j and j!= k:
                triplets.append((i, j, k))
    return triplets

print(find_triplets(108))

[(1, 2, 54), (1, 3, 36), (1, 4, 27), (1, 6, 18), (1, 9, 12), (2, 3, 18), (2, 6, 9), (3, 4, 9)]


# You.com

You.com produces a correct solution, with some nice optimizations that make it *O*(*n*<sup>5/6</sup>), whereas most of the solutions are *O*(*n*<sup>2</sup>). This means it can handle a 14-digit product in about a second of run time, whereas the HuggingChat version (for example) can only handle 5-digit products in that time.

In [11]:
def find_triplets(product):
    triplets = []
    for i in range(1, int(product ** (1/3)) + 1):  # The cube root of the product is the maximum possible value for i
        if product % i == 0:
            for j in range(i + 1, int((product / i) ** 0.5) + 1):  # The square root of the product divided by i is the maximum possible value for j
                if (product / i) % j == 0:
                    k = product // (i * j)
                    if k > j:  # Ensure the integers are distinct
                        triplets.append((i, j, k))
    return triplets

triplets = find_triplets(108)
print(triplets)

[(1, 2, 54), (1, 3, 36), (1, 4, 27), (1, 6, 18), (1, 9, 12), (2, 3, 18), (2, 6, 9), (3, 4, 9)]
